# Results inspection for EV dispatch
Inspect and visualize patterns from EV dispatch optimization

## Package imports

In [ ]:
import pandas as pd
from pommesevaluation.electric_vehicles_dispatch import draw_weekly_plot

## Notebook and workflow settings

In [ ]:
# Simulation settings
time_frame_in_years = 26
freq = "1H"
multiplier = {
    "1H": 1,
    "4H": 4,
    "8H": 8,
}
fuel_cost_pathway = "NZE"
emissions_cost_pathway = "long-term"
impose_investment_maxima = False
dr_scenario = "50"

# Paths to data needed to be converted
path_model_inputs = "./model_inputs/pommesinvest/"
path_model_results = "./model_results/pommesinvest/"

# Map corresponding flex options scenario (values) to dr scenarios (keys)
dr_scenarios = {
    "none": "50",
    "5": "5", 
    "50": "50",
    "95": "95"
}

if impose_investment_maxima:
    annual_investment_limits = ""
else:
    annual_investment_limits = "_no_annual_limit"
file_names_dispatch_results = {
    scen: (
        f"investment_LP_start-2020-01-01_{time_frame_in_years}"
        f"-years_simple_freq_{freq}_with_dr_{scen}_"
        f"fuel_price-{fuel_cost_pathway}_"
        f"co2_price-{emissions_cost_pathway}{annual_investment_limits}_production.csv"
    ) for scen in dr_scenarios
}
file_names_dispatch_results["none"] = (
    f"investment_LP_start-2020-01-01_{time_frame_in_years}"
    f"-years_simple_freq_{freq}_no_dr_50_"
    f"fuel_price-{fuel_cost_pathway}_"
    f"co2_price-{emissions_cost_pathway}{annual_investment_limits}_production.csv"
)
dispatch_multi_index = False

file_name_electric_vehicles = f"components_electric_vehicles_{dr_scenario}.csv"
file_name_electric_vehicles_ts = f"electric_vehicles_ts_{dr_scenario}.csv"

## Read in and filter electric vehicles dispatch results

In [ ]:
if dispatch_multi_index:
    header = [0, 1]
else:
    header = 0

dispatch_results = pd.read_csv(
    f"{path_model_results}{file_names_dispatch_results[dr_scenario]}", index_col=0, header=header
)
ev_dispatch = dispatch_results[[
    col for col in dispatch_results.columns if "ev_" in col or "_ev" in col
]]

## Calculate and visualize dispatch
* Derive net flow for controlled charging (negative values: feed-in into the grid)
* Derive net storage operation (negative values: discharging)
* Visualize weekly dispatch patterns for 2020

In [ ]:
# Extract electricity consumptions
ev_dispatch["cc_bidirectional_net_flow"] = ev_dispatch[
    "(('DE_bus_el', 'storage_ev_cc_bidirectional'), 'flow')"
] - ev_dispatch[
    "(('transformer_ev_cc_bidirectional_feedback', 'DE_bus_el'), 'flow')"
]
ev_dispatch["cc_unidirectional_net_flow"] = ev_dispatch[
    "(('DE_bus_el', 'storage_ev_cc_unidirectional'), 'flow')"
]

# Extract net storage operations
ev_dispatch["net_storage_cc_bidirectional"] = ev_dispatch[
    "(('DE_bus_el', 'storage_ev_cc_bidirectional'), 'flow')"
] - ev_dispatch[
    "(('storage_ev_cc_bidirectional', 'DE_bus_ev_cc_bidirectional'), 'flow')"
]
ev_dispatch["net_storage_cc_unidirectional"] = ev_dispatch[
    "(('DE_bus_el', 'storage_ev_cc_unidirectional'), 'flow')"
] - ev_dispatch[
    "(('storage_ev_cc_unidirectional', 'DE_bus_ev_cc_unidirectional'), 'flow')"
]

# Extract demand for driving
ev_dispatch["bidirectional_driving_demand"] = ev_dispatch[
    "(('DE_bus_ev_cc_bidirectional', 'sink_ev_driving_cc_bidirectional'), 'flow')"
]
ev_dispatch["unidirectional_driving_demand"] = ev_dispatch[
    "(('DE_bus_ev_cc_bidirectional', 'sink_ev_driving_cc_bidirectional'), 'flow')"
]

# Extract EV storage level patterns
ev_dispatch["storage_cc_bidirectional_cumsum"] = ev_dispatch["net_storage_cc_bidirectional"].cumsum()
ev_storage_initial = -ev_dispatch["storage_cc_bidirectional_cumsum"].min()
ev_dispatch["storage_cc_bidirectional_cumsum"] += ev_storage_initial
ev_capacity_bidirectional = (
    ev_dispatch["storage_cc_bidirectional_cumsum"].max() - ev_dispatch["storage_cc_bidirectional_cumsum"].min()
)

ev_dispatch["storage_cc_unidirectional_cumsum"] = ev_dispatch["net_storage_cc_unidirectional"].cumsum()
ev_storage_initial = -ev_dispatch["storage_cc_bidirectional_cumsum"].min()
ev_dispatch["storage_cc_bidirectional_cumsum"] += ev_storage_initial
ev_capacity_unidirectional = (
    ev_dispatch["storage_cc_bidirectional_cumsum"].max() - ev_dispatch["storage_cc_bidirectional_cumsum"].min()
)

In [ ]:
ev_dispatch.max()

In [ ]:
draw_weekly_plot(
    ev_dispatch[[
        "cc_bidirectional_net_flow", 
        "bidirectional_driving_demand",
        "cc_unidirectional_net_flow", 
        "unidirectional_driving_demand"
    ]], 2020,
    file_name="ev_electricity_consumption"
)

In [ ]:
draw_weekly_plot(
    ev_dispatch[[
        "net_storage_cc_bidirectional", 
        "net_storage_cc_unidirectional",
    ]], 2020,
    file_name="ev_storage_operation"
)

## Read in parameterization and derive parameter data from it

In [ ]:
electric_vehicles = pd.read_csv(
    f"{path_model_inputs}{file_name_electric_vehicles}", index_col=0
)
electric_vehicles_ts = pd.read_csv(
    f"{path_model_inputs}{file_name_electric_vehicles_ts}", index_col=0
)

# Calculate usable storage capacity from lower and upper bounds (max - min)
usable_storage_capacity = (electric_vehicles.at["storage_ev_cc_bidirectional", "nominal_value"] * (
    electric_vehicles_ts.loc[electric_vehicles_ts.index.str[:4] == "2020"].max().loc["soc_upper_cc_bidirectional"]
) - electric_vehicles.at["storage_ev_cc_bidirectional", "nominal_value"] * (
    electric_vehicles_ts.loc[electric_vehicles_ts.index.str[:4] == "2020"].min().loc["soc_lower_cc_bidirectional"]
))